In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//table[@class="table c-table c-table--split"]/tbody/tr')
                
                # = = = = = = = = = = = = = = =
                
                list_th = [th.strip() for th in html.xpath('//table[@class="table c-table c-table--striped"]/thead/tr/th/text()')]
                
                # = = = = = = = = = = = = = = =
                
                list_tr_2 = html.xpath('//table[@class="table c-table c-table--striped"]/tbody/tr')

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'序号': int(input_.loc[a, 'No.']),
                                        'Url': input_.loc[a, 'Url'],
                                        'Kit_序号': [i+1 for i in range(len(list_tr_2))]})

                for tr in list_tr:
                    df_temp[tr.xpath('./th/text()')[0].strip()] = '' if len(tr.xpath('./td/text()')) == 0 else tr.xpath('./td/text()')[0].strip()

                for i in range(len(list_th)):
                    list_td = [tr.xpath('./td[' + str(i+1) + ']')[0] for tr in list_tr_2]
                    list_td = ['' if td.text == None else td.text.strip() for td in list_td]
                    df_temp['Kit_' + list_th[i]] = list_td

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号', 'Kit_序号'], ascending=[True, True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：3

[ok] - https://www.gates.com/gb/en/power-transmission/power-transmission-kits/micro-v-kits.p.7884-000000-000000.v.8542-20533.html
[尝试次数：1] - [剩余数量：0] - [当前时间：20:58:34]

[ok] - https://www.gates.com/gb/en/power-transmission/power-transmission-kits/micro-v-kits.p.7884-000000-000000.v.7884-22875.html
[尝试次数：1] - [剩余数量：0] - [当前时间：20:58:35]

[ok] - https://www.gates.com/gb/en/power-transmission/power-transmission-kits/micro-v-kits.p.7884-000000-000000.v.7884-22551.html
[尝试次数：1] - [剩余数量：0] - [当前时间：20:58:38]

输出ing...

Done ~
